In [5]:
import pandas as pd
from datetime import datetime
import os

In [44]:
raw_history_df = pd.read_csv('./data/metadata/shutdowns_master.csv')
raw_prez_df = pd.read_csv('./data/metadata/us_presidents_britannica.csv')

In [59]:
history_df = raw_history_df[raw_history_df['shutdown_flag']]
history_df['start_year'] = history_df['start_date'].str[:4].astype(int)
prez_df = raw_prez_df.drop(columns=['president_number', 'birthplace', 'term_of_office'])
prez_df['term_start'] = prez_df['term_start'].astype(int)
prez_df['term_end'] = prez_df['term_end'].astype(int)


/var/folders/s9/b35x2lvd0zl66546s25wwsjc0000gn/T/ipykernel_59379/1734163971.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_df['start_year'] = history_df['start_date'].str[:4].astype(int)


In [53]:
history_df.head()

,fiscal_year,funding_gap_start,duration_days,funding_gap_end,shutdown_flag,restoring_legislation,date_range,start_date,end_date,start_year
6,1982,"November 20, 1981",2,"November 23, 1981",True,H.J. Res. 368\nPub. L. 97-85\n95 Stat. 1098,"November 20, 1981 – November 23, 1981",1981-11-20,1981-11-23,1981
7,1983,"September 30, 1982",1,"October 2, 1982",True,H.J. Res. 599\nPub. L. 97-276\n96 Stat. 1186,"September 30, 1982 – October 2, 1982",1982-09-30,1982-10-02,1982
11,1985,"October 3, 1984",1,"October 5, 1984",True,H.J. Res. 656\nPub. L. 98-453\n98 Stat. 1731,"October 3, 1984 – October 5, 1984",1984-10-03,1984-10-05,1984
12,1987,"October 16, 1986",1,"October 18, 1986",True,H.J. Res. 73812\n\nPub. L. 99-500\n100 Stat. 1...,"October 16, 1986 – October 18, 1986",1986-10-16,1986-10-18,1986
14,1991,"October 5, 1990",3,"October 9, 1990",True,H.J. Res. 666\nPub. L. 101-412\n104 Stat. 894,"October 5, 1990 – October 9, 1990",1990-10-05,1990-10-09,1990


In [55]:
prez_df.head()

,name,party,term_start,term_end
0,George Washington,Federalist,1789,1797
1,John Adams,Federalist,1797,1801
2,Thomas Jefferson,Democratic-Republican,1801,1809
3,James Madison,Democratic-Republican,1809,1817
4,James Monroe,Democratic-Republican,1817,1825


In [62]:
history_df = (
    history_df.assign(key=1)
    .merge(prez_df.assign(key=1), on="key")
    .query("start_year >= term_start and start_year <= term_end")
    .drop(columns="key")
).reset_index(drop=True)

In [65]:
history_df.rename(columns={'name':'president'}, inplace=True)
history_df.drop(columns=['start_year'])

,fiscal_year,funding_gap_start,duration_days,funding_gap_end,shutdown_flag,restoring_legislation,date_range,start_date,end_date,president,party,term_start,term_end
0,1982,"November 20, 1981",2,"November 23, 1981",True,H.J. Res. 368\nPub. L. 97-85\n95 Stat. 1098,"November 20, 1981 – November 23, 1981",1981-11-20,1981-11-23,Jimmy Carter,Democratic,1977,1981
1,1982,"November 20, 1981",2,"November 23, 1981",True,H.J. Res. 368\nPub. L. 97-85\n95 Stat. 1098,"November 20, 1981 – November 23, 1981",1981-11-20,1981-11-23,Ronald Reagan,Republican,1981,1989
2,1983,"September 30, 1982",1,"October 2, 1982",True,H.J. Res. 599\nPub. L. 97-276\n96 Stat. 1186,"September 30, 1982 – October 2, 1982",1982-09-30,1982-10-02,Ronald Reagan,Republican,1981,1989
3,1985,"October 3, 1984",1,"October 5, 1984",True,H.J. Res. 656\nPub. L. 98-453\n98 Stat. 1731,"October 3, 1984 – October 5, 1984",1984-10-03,1984-10-05,Ronald Reagan,Republican,1981,1989
4,1987,"October 16, 1986",1,"October 18, 1986",True,H.J. Res. 73812\n\nPub. L. 99-500\n100 Stat. 1...,"October 16, 1986 – October 18, 1986",1986-10-16,1986-10-18,Ronald Reagan,Republican,1981,1989
5,1991,"October 5, 1990",3,"October 9, 1990",True,H.J. Res. 666\nPub. L. 101-412\n104 Stat. 894,"October 5, 1990 – October 9, 1990",1990-10-05,1990-10-09,George Bush,Republican,1989,1993
6,1996,"November 13, 1995",5,"November 19, 1995",True,H.R. 2020\nPub. L. 104-52\n109 Stat. 468\n\nH....,"November 13, 1995 – November 19, 1995",1995-11-13,1995-11-19,Bill Clinton,Democratic,1993,2001
7,1996,"December 15, 1995",21,"January 6, 1996",True,H.J. Res. 134\nPub. L. 104-94\n110 Stat. 25,"December 15, 1995 – January 6, 1996",1995-12-15,1996-01-06,Bill Clinton,Democratic,1993,2001
8,2014,"September 30, 2013",16,"October 17, 2013",True,H.R. 2775\nPub. L. 113-46\n127 Stat. 558,"September 30, 2013 – October 17, 2013",2013-09-30,2013-10-17,Barack Obama,Democratic,2009,2017
9,2018,"January 19, 2018",2,"January 22, 2018",True,H.R. 195\nPub. L. 115-120\n132 Stat. 28,"January 19, 2018 – January 22, 2018",2018-01-19,2018-01-22,Donald Trump,Republican,2017,2021
